In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/MyDrive/spundsteif/spundsteif'
!pip3 install -e . -q
!pip3 install nbdev -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/spundsteif/'
/content/drive/MyDrive/spundsteif/spundsteif
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/drive/My Drive/spundsteif/spundsteif


In [ ]:
# default_exp sensors

# Sensors
> This library transforms raw data from sensor measurements into cleaned database.  
> Some functions aiming a graphical check of measurements are also available.  
--- 
**Inputs:**  
+ un ensemble de fichiers brutes des stations de mesures sous format ASCI: `'1_messung_xxx.ASC'`; 
+ un fichier CSV des relevés manuels `'df_hand.csv'`; 
+ *(#TO DO) `'df_evolution.csv'` contient l'évolution de l'état de fonctionnement des capteurs ;*

**Outputs:**  
+  `df_messung` : base de données regroupant les valeurs renvoyées par les capteurs de pression (*EDS*), jauges de déformation (*DMS*) et thermomètres, relevées manuellement **et** de manière semi-continue

----
**Metadata:**  
+ les caractéristiques géométriques de l'installation des capteurs, informations contenues dans `'df_proprietes.csv'` ;


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
%ls

core.py  __init__.py  _nbdev.py  __pycache__/  sensors.py


In [ ]:
#hide
!nbdev_build_lib

Converted 01_sensors.ipynb.
Converted index.ipynb.
The following paths are ignored by one of your .gitignore files:
spundsteif/__pycache__
Use -f if you really want to add them.
On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
	deleted:    ../00_core.ipynb

Untracked files:
	../01_sensors.ipynb
	../Makefile
	../docs/_config.yml
	../docs/_data/

no changes added to commit
fatal: could not read Username for 'https://github.com': No such device or address
